# Fasta to LMDB

This notebook provides a simple utility to convert from a `fasta` file to `lmdb` (lightning memory-mapped database), which provides high performance reads and writes for training a model compared to loading data from a `fasta` file

Specifically, we run this notebook for the three `fasta` files created in notebook 1, for all sequences with a sequence length < 6000.

This notebook is necessary to run the TAPE models on subcellular location, which is shown in notebook 3 on Google colab.

In [31]:
from collections import defaultdict
from pathlib import Path
import pickle
import os
import sys

from Bio import SeqIO
import lmdb

In [32]:
FASTA_TRAIN = list()
FASTA_VALID = list()
FASTA_TEST = list()

In [33]:
# Function to read and parse a fasta file and extract the sequence ID, label, and sequence
def read_fasta(fasta_path, fasta_list):
    train_file = f'../{fasta_path}'
    for sequence in SeqIO.parse(str(train_file), 'fasta'):
        id = sequence.id
        label = sequence.description.split()[1]
        seq = str(sequence.seq)
        fasta_list.append({
            'id': id,
            'label': label,
            'primary': seq
        })

In [34]:
# Function to write a key/value pair to a local LMDB database
def write_record_to_lmdb(db, key, value):
    """
    Write (key,value) to db
    """
    success = False
    while not success:
        txn = db.begin(write=True)
        try:
            txn.put(key, value)
            txn.commit()
            success = True
        except lmdb.MapFullError:
            txn.abort()
            # double the map_size
            curr_limit = db.info()['map_size']
            new_limit = curr_limit * 2
            print(f'Doubling LMDB map size to {new_limit}')
            db.set_mapsize(new_limit)

In [35]:
# Given the sequence ID, label, and sequence for each protein, write the records to an LMDB file
def write_sc_lmdb(split, fasta_list):
    map_size = sys.getsizeof(pickle.dumps(fasta_list)) * 10
    env = lmdb.open((f'../data/deeploc/deeploc_{split}.lmdb'), map_size=map_size)
    for i in range(len(fasta_list)):
        write_record_to_lmdb(env, str(i).encode(), pickle.dumps(fasta_list[i]))
    write_record_to_lmdb(env, b'num_examples', pickle.dumps(len(fasta_list)))

In [36]:
# Utility function to read sequences from an LMDB file to verify that the files were converted correctly
def read_sc_lmdb(split, verbose_flag=False):
    env = lmdb.open(f'../data/deeploc/deeploc_{split}.lmdb')
    with env.begin(write=False) as txn:
        num_examples = pickle.loads(txn.get(b'num_examples'))
        print(f'{split} has num_examples={num_examples}')

    if verbose_flag:
        label_counts = defaultdict(int)
        with env.begin(write=False) as txn:
            for index in range(num_examples):
                item = pickle.loads(txn.get(str(index).encode()))
                if 'id' not in item:
                    item['id'] = str(index)
                label_counts[item['label']] += 1

        print(label_counts)

In [37]:
# Create LMDB files for train/valid/test
MAX_SEQ_LENGTH = 6000

# Train
read_fasta(f'data/deeploc_train_{MAX_SEQ_LENGTH}.fasta', FASTA_TRAIN)
write_sc_lmdb('train', FASTA_TRAIN)
read_sc_lmdb('train', True)

# Validation
read_fasta(f'data/deeploc_valid_{MAX_SEQ_LENGTH}.fasta', FASTA_VALID)
write_sc_lmdb('valid', FASTA_VALID)
read_sc_lmdb('valid', True)

# Test
read_fasta(f'data/deeploc_test_{MAX_SEQ_LENGTH}.fasta', FASTA_TEST)
write_sc_lmdb('test', FASTA_TEST)
read_sc_lmdb('test', True)

train has num_examples=9974
defaultdict(<class 'int'>, {'1': 1830, '8': 545, '5': 1087, '6': 2911, '9': 1421, '0': 960, '3': 257, '2': 620, '7': 112, '4': 231})
valid has num_examples=1109
defaultdict(<class 'int'>, {'6': 324, '0': 107, '9': 158, '8': 60, '5': 121, '2': 69, '1': 203, '7': 12, '4': 26, '3': 29})
test has num_examples=2773
defaultdict(<class 'int'>, {'0': 273, '1': 508, '2': 173, '3': 70, '4': 64, '5': 302, '6': 808, '7': 30, '8': 152, '9': 393})
